This is just some messing around that I am doing with the stanza
and the TF-IDF summarization tutorial.


In [ ]:
import stanza
from collections import Counter
from string import punctuation

In [ ]:
# Download English Language Model
stanza.download('en')

2020-11-05 15:59:20 INFO: Downloading default packages for language: en (English)...
2020-11-05 15:59:41 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
# Initiate an English LM stanza pipeline
nlp = stanza.Pipeline('en')

2020-11-05 15:59:47 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-11-05 15:59:47 INFO: Use device: cpu
2020-11-05 15:59:47 INFO: Loading: tokenize
2020-11-05 15:59:47 INFO: Loading: pos
2020-11-05 15:59:48 INFO: Loading: lemma
2020-11-05 15:59:48 INFO: Loading: depparse
2020-11-05 15:59:49 INFO: Loading: sentiment
2020-11-05 15:59:51 INFO: Loading: ner
2020-11-05 15:59:51 INFO: Done loading processors!


#### From Tutorial

https://medium.com/better-programming/extractive-text-summarization-using-spacy-in-python-88ab96d1fd97


TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)

TF: Term Frequency — Measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear many more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length, such as the total number of terms in the document, as a way of normalization.

IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

IDF: Inverse Document Frequency — Measures how important a term is. While computing the term frequency, all terms are considered equally important. However, it is known that certain terms may appear a lot of times but have little importance in the document. We usually term these words stopwords. For example: is, are, they, and so on.

In [ ]:
def top_sentence(text, limit):
    """Function to read in a text and return a summary made from
    the top (n=limit) sentence constituents
    PARAMS: text (str) - book to be summarized
            limit (int) - the number of the top sentences to use for the summary
    RETURNS: (str) - the summary of the document, represented as the most
                     important sentences by TF-IDF
    """
    # Capture keywords from text
    keyword = []        # initiate keyword list
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']   # Are these the only POS we want?
    doc = nlp(text.lower())     # TO-DO: convert to stanza # normalize text and build stanza object
    for token in doc:   # iterate through all document tokens
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation): # TO-DO: convert to stanza # ignore punctuation and stop_words
            continue
        if(token.pos_ in pos_tag): # TO-DO: convert to stanza   # check against allowable tagset
            keyword.append(token.text)          # append keyword to storage

    # Calculate a normalixed value for each keyword
    freq_word = Counter(keyword)            # count keywords
    max_freq = Counter(keyword).most_common(1)[0][1]   # Capture frequency of most common word
    for w in freq_word:
        freq_word[w] = (freq_word[w]/max_freq)  # use most common freq to normalize all

    # Calculate a cumulative normalized value for each sentence
    sent_strength={}
    for sent in doc.sents:  # TO-DO: convert to stanza # iterate through sentences
        for word in sent:   # iterate through words in a sentence
            if word.text in freq_word.keys():   # TO-DO: convert to stanza # check word against keywords
                if sent in sent_strength.keys():  # Check if we have this key
                    sent_strength[sent]+=freq_word[word.text]   # increment if we do
                else:
                    sent_strength[sent]=freq_word[word.text]    # create if we don't

    # Aggregate Summary
    summary = []        # initiate empty list for sentences
    
    sorted_x = sorted(sent_strength.items(), key=lambda kv: kv[1], reverse=True) # sort sentences by weight

    counter = 0     # initialize counter variable
    for i in range(len(sorted_x)):      # iterate through list of sorted sentences
        summary.append(str(sorted_x[i][0]).capitalize())        # append the capitalized sentence to the list
        counter += 1        # increment counter var
        if(counter >= limit):       # early stop at the stated limit
            break
            
    return ' '.join(summary)        # return string of top sentences